In [5]:
from curl_cffi import requests
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import talib as ta
import seaborn as sns
sns.set_theme(style="whitegrid")
import pickle
import duckdb as db

In [6]:
window = 20

In [ ]:
with open('../../data/n200_5yr_v3.pickle', 'rb') as f:
    stock_data = pickle.load(f)

with open('../../data/nifty_data.pkl', 'rb') as f:
    index_data = pickle.load(f)

with open('../../data/nifty_regimes_m.pkl', 'rb') as f:
    regime = pickle.load(f)
    
n200 = pd.read_csv("../../data/ind_nifty200list.csv")

In [8]:
index_data = index_data.reset_index().rename(columns={'index': 'Date', 'Close': 'Index_Close'})[['Date', 'Index_Close']]

In [9]:
index_data["Date"] = index_data['Date'].dt.tz_localize(None)
index_data["index_ret_1d"]= index_data['Index_Close'].pct_change(1)
# index_data["var"] = index_data['index_ret_1d'].rolling(window).var()
index_data["index_vol"] = index_data["index_ret_1d"].rolling(window).std()

In [10]:
index_data=index_data.set_index("Date")

In [11]:
full_idx = pd.date_range(index_data.index.min(), index_data.index.max(), freq="D")
index_data = index_data.reindex(full_idx)

In [12]:
index_data=index_data.bfill().ffill()
index_data

,Index_Close,index_ret_1d,index_vol
2007-09-17,4494.649902,0.011469,0.018438
2007-09-18,4546.200195,0.011469,0.018438
2007-09-19,4732.350098,0.040946,0.018438
2007-09-20,4747.549805,0.003212,0.018438
2007-09-21,4837.549805,0.018957,0.018438
...,...,...,...
2025-09-15,25069.199219,-0.001784,0.005349
2025-09-16,25239.099609,0.006777,0.005105
2025-09-17,25330.250000,0.003611,0.005087
2025-09-18,25423.599609,0.003685,0.005110


In [13]:
index_data=index_data.reset_index()
index_data=index_data.rename(columns={'index': 'Date'})

In [14]:
index_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6578 entries, 0 to 6577
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          6578 non-null   datetime64[ns]
 1   Index_Close   6578 non-null   float64       
 2   index_ret_1d  6578 non-null   float64       
 3   index_vol     6578 non-null   float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 205.7 KB


In [15]:
stock_data.head()

,Date,Open,High,Low,Close,Volume,Symbol
0,2015-09-16,977.616089,989.007507,951.472412,957.844666,46055,ABB
1,2015-09-18,977.616089,987.392639,957.757324,978.969055,129547,ABB
2,2015-09-21,973.295166,991.539001,964.566040,966.486450,34553,ABB
3,2015-09-22,979.361938,979.361938,960.201599,962.602051,13607,ABB
4,2015-09-23,960.201355,964.478638,944.838135,959.677612,15296,ABB


In [16]:
# count number of zeroes in each column
(stock_data==0).sum()

Date        0
Open        0
High        0
Low         0
Close       0
Volume    502
Symbol      0
dtype: Int64

In [17]:
# replace zeroes with NaN
stock_data = stock_data.replace(0, np.nan)

In [18]:
stock_data

,Date,Open,High,Low,Close,Volume,Symbol
0,2015-09-16,977.616089,989.007507,951.472412,957.844666,46055.0,ABB
1,2015-09-18,977.616089,987.392639,957.757324,978.969055,129547.0,ABB
2,2015-09-21,973.295166,991.539001,964.566040,966.486450,34553.0,ABB
3,2015-09-22,979.361938,979.361938,960.201599,962.602051,13607.0,ABB
4,2015-09-23,960.201355,964.478638,944.838135,959.677612,15296.0,ABB
...,...,...,...,...,...,...,...
430042,2025-09-09,1010.000000,1038.699951,1009.200012,1035.599976,1228793.0,ZYDUSLIFE
430043,2025-09-10,1040.550049,1049.849976,1033.000000,1039.599976,1086228.0,ZYDUSLIFE
430044,2025-09-11,1039.599976,1048.000000,1030.300049,1043.599976,537049.0,ZYDUSLIFE
430045,2025-09-12,1048.300049,1054.599976,1042.000000,1044.750000,559927.0,ZYDUSLIFE


In [19]:
# stock_data["Volume"] = stock_data.groupby('Symbol', group_keys=False)["Volume"].apply(lambda group: group.interpolate())
stock_data["Volume"] = stock_data.groupby('Symbol')["Volume"].transform(lambda x: x.interpolate())

In [20]:
(stock_data==0).sum()

Date      0
Open      0
High      0
Low       0
Close     0
Volume    0
Symbol    0
dtype: Int64

In [54]:
stock_data.groupby('Symbol').apply(lambda g: ta.OBV(g["Close"], g["Volume"]), include_groups=False).reset_index(level=0, drop=True)

0             46055.0
1            175602.0
2            141049.0
3            127442.0
4            112146.0
             ...     
430042    682849209.0
430043    683935437.0
430044    684472486.0
430045    685032413.0
430046    684640908.0
Length: 430047, dtype: float64

In [50]:
stock_data.groupby('Symbol').apply(lambda g: ta.OBV(g["Close"], g["Volume"]), include_groups=False)

Symbol           
ABB        0             46055.0
           1            175602.0
           2            141049.0
           3            127442.0
           4            112146.0
                        ...     
ZYDUSLIFE  430042    682849209.0
           430043    683935437.0
           430044    684472486.0
           430045    685032413.0
           430046    684640908.0
Length: 430047, dtype: float64

In [51]:
stock_data["stock_ret_1d"] = stock_data.groupby('Symbol')['Close'].transform(lambda x: x.pct_change(1))
stock_data["vol_norm"] = stock_data.groupby('Symbol')["Volume"].transform(lambda x: x / x.rolling(window).mean())
stock_data["obv"] = stock_data.groupby('Symbol').apply(lambda g: ta.OBV(g["Close"], g["Volume"]), include_groups=False).reset_index(level=0, drop=True)

In [55]:
stock_data

,Date,Open,High,Low,Close,Volume,Symbol,stock_ret_1d,vol_norm,obv
0,2015-09-16,977.616089,989.007507,951.472412,957.844666,46055.0,ABB,NaN,NaN,46055.0
1,2015-09-18,977.616089,987.392639,957.757324,978.969055,129547.0,ABB,0.022054,NaN,175602.0
2,2015-09-21,973.295166,991.539001,964.566040,966.486450,34553.0,ABB,-0.012751,NaN,141049.0
3,2015-09-22,979.361938,979.361938,960.201599,962.602051,13607.0,ABB,-0.004019,NaN,127442.0
4,2015-09-23,960.201355,964.478638,944.838135,959.677612,15296.0,ABB,-0.003038,NaN,112146.0
...,...,...,...,...,...,...,...,...,...,...
430042,2025-09-09,1010.000000,1038.699951,1009.200012,1035.599976,1228793.0,ZYDUSLIFE,0.025702,0.974756,682849209.0
430043,2025-09-10,1040.550049,1049.849976,1033.000000,1039.599976,1086228.0,ZYDUSLIFE,0.003863,0.855775,683935437.0
430044,2025-09-11,1039.599976,1048.000000,1030.300049,1043.599976,537049.0,ZYDUSLIFE,0.003848,0.447328,684472486.0
430045,2025-09-12,1048.300049,1054.599976,1042.000000,1044.750000,559927.0,ZYDUSLIFE,0.001102,0.469458,685032413.0


In [56]:
merged = pd.merge(stock_data, index_data, on='Date', how='left')

In [57]:
merged

,Date,Open,High,Low,Close,Volume,Symbol,stock_ret_1d,vol_norm,obv,Index_Close,index_ret_1d,index_vol
0,2015-09-16,977.616089,989.007507,951.472412,957.844666,46055.0,ABB,NaN,NaN,46055.0,7899.149902,0.008947,0.018447
1,2015-09-18,977.616089,987.392639,957.757324,978.969055,129547.0,ABB,0.022054,NaN,175602.0,7981.899902,0.010476,0.018509
2,2015-09-21,973.295166,991.539001,964.566040,966.486450,34553.0,ABB,-0.012751,NaN,141049.0,7977.100098,-0.000601,0.018448
3,2015-09-22,979.361938,979.361938,960.201599,962.602051,13607.0,ABB,-0.004019,NaN,127442.0,7812.000000,-0.020697,0.013498
4,2015-09-23,960.201355,964.478638,944.838135,959.677612,15296.0,ABB,-0.003038,NaN,112146.0,7845.950195,0.004346,0.013370
...,...,...,...,...,...,...,...,...,...,...,...,...,...
430042,2025-09-09,1010.000000,1038.699951,1009.200012,1035.599976,1228793.0,ZYDUSLIFE,0.025702,0.974756,682849209.0,24868.599609,0.003853,0.005733
430043,2025-09-10,1040.550049,1049.849976,1033.000000,1039.599976,1086228.0,ZYDUSLIFE,0.003863,0.855775,683935437.0,24973.099609,0.004202,0.005469
430044,2025-09-11,1039.599976,1048.000000,1030.300049,1043.599976,537049.0,ZYDUSLIFE,0.003848,0.447328,684472486.0,25005.500000,0.001297,0.005352
430045,2025-09-12,1048.300049,1054.599976,1042.000000,1044.750000,559927.0,ZYDUSLIFE,0.001102,0.469458,685032413.0,25114.000000,0.004339,0.005313


In [58]:
with open("../../data/index_stocked_merged.pkl", 'wb') as f:
    pickle.dump(merged, f)

# with open("../../data/index_stocked_merged.pkl", 'rb') as f:
#     merged = pickle.load(f)

In [80]:
regime=regime.reset_index().rename(columns={"index": "Date"})

In [38]:
merged.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Symbol',
       'stock_ret_1d', 'Index_Close', 'index_ret_1d', 'index_vol'],
      dtype='object')

In [59]:
merged

,Date,Open,High,Low,Close,Volume,Symbol,stock_ret_1d,vol_norm,obv,Index_Close,index_ret_1d,index_vol
0,2015-09-16,977.616089,989.007507,951.472412,957.844666,46055.0,ABB,NaN,NaN,46055.0,7899.149902,0.008947,0.018447
1,2015-09-18,977.616089,987.392639,957.757324,978.969055,129547.0,ABB,0.022054,NaN,175602.0,7981.899902,0.010476,0.018509
2,2015-09-21,973.295166,991.539001,964.566040,966.486450,34553.0,ABB,-0.012751,NaN,141049.0,7977.100098,-0.000601,0.018448
3,2015-09-22,979.361938,979.361938,960.201599,962.602051,13607.0,ABB,-0.004019,NaN,127442.0,7812.000000,-0.020697,0.013498
4,2015-09-23,960.201355,964.478638,944.838135,959.677612,15296.0,ABB,-0.003038,NaN,112146.0,7845.950195,0.004346,0.013370
...,...,...,...,...,...,...,...,...,...,...,...,...,...
430042,2025-09-09,1010.000000,1038.699951,1009.200012,1035.599976,1228793.0,ZYDUSLIFE,0.025702,0.974756,682849209.0,24868.599609,0.003853,0.005733
430043,2025-09-10,1040.550049,1049.849976,1033.000000,1039.599976,1086228.0,ZYDUSLIFE,0.003863,0.855775,683935437.0,24973.099609,0.004202,0.005469
430044,2025-09-11,1039.599976,1048.000000,1030.300049,1043.599976,537049.0,ZYDUSLIFE,0.003848,0.447328,684472486.0,25005.500000,0.001297,0.005352
430045,2025-09-12,1048.300049,1054.599976,1042.000000,1044.750000,559927.0,ZYDUSLIFE,0.001102,0.469458,685032413.0,25114.000000,0.004339,0.005313


In [60]:
def add_indicators(df: pd.DataFrame, window:int=20) -> pd.DataFrame:
    """
    Add stock-only indicators + stock-index relationship features
    for a stacked dataframe of multiple stocks.
    
    Required columns:
    ['Symbol','Date','Open','High','Low','Close','Volume','Index_Close']
    """

    df = df.copy()

    grouped = df.groupby("Symbol", group_keys=False)

    # ------------------------------
    # 1. Trend / Moving Averages
    # ------------------------------
    df["EMA_9"]   = grouped["Close"].transform(lambda x: ta.EMA(x, 9))
    df["EMA_30"]  = grouped["Close"].transform(lambda x: ta.EMA(x, 30))
    df["EMA_100"] = grouped["Close"].transform(lambda x: ta.EMA(x, 100))

    # Price ratios
    df["ema_9_ratio"]   = df["Close"] / df["EMA_9"]
    df["ema_30_ratio"]  = df["Close"] / df["EMA_30"]
    df["ema_100_ratio"] = df["Close"] / df["EMA_100"]

    # ------------------------------
    # 2. Directional Indicators
    # ------------------------------
    df["plus_di"]  = grouped.apply(lambda g: ta.PLUS_DI(g["High"], g["Low"], g["Close"], timeperiod=14), include_groups=False)
    df["minus_di"] = grouped.apply(lambda g: ta.MINUS_DI(g["High"], g["Low"], g["Close"], timeperiod=14), include_groups=False)
    df["adx"]      = grouped.apply(lambda g: ta.ADX(g["High"], g["Low"], g["Close"], timeperiod=14), include_groups=False)


    # ------------------------------
    # 3. Oscillators / Momentum
    # ------------------------------
    df["rsi"] = grouped["Close"].transform(lambda x: ta.RSI(x, 14))
    for n in [3, 7, 14, 30]:
        df[f"roc_{n}"] = grouped["Close"].transform(lambda x: ta.ROC(x, n))


    # ------------------------------
    # 4. ATR-based Volatility
    # ------------------------------
    for n in [3, 7, 14, 30]:
        df[f"atr_{n}"] = grouped.apply(lambda g: ta.ATR(g["High"], g["Low"], g["Close"], timeperiod=n), include_groups=False)
        df[f"atr_{n}"] = df[f"atr_{n}"] / df["Close"]

    # ------------------------------
    # 5. Returns & Rolling Volatility
    # ------------------------------
    # df["stock_ret_1d"]  = grouped["Close"].transform(lambda x: x.pct_change(1))

    for n in [3, 7, 14, 30]:
        df[f"stock_ret_{n}d"] = grouped["Close"].transform(lambda x: x.pct_change(n))

    for n in [3, 7, 14, 30]:
        df[f"vol_{n}d"] = grouped["stock_ret_1d"].transform(lambda x: x.rolling(n).std())

    # ------------------------------
    # 6. Rolling Highs normalized
    # ------------------------------
    for n in [3, 7, 14, 30]:
        df[f"{n}_days_high"] = grouped["High"].transform(lambda x: x.rolling(n).max()) / df["Close"]

    # ------------------------------
    # 7. MACD (normalized by Close)
    # ------------------------------
    def macd_apply(x):
        macd, macd_signal, macd_hist = ta.MACD(x, fastperiod=12, slowperiod=26, signalperiod=9)
        return pd.DataFrame({"macd": macd, "macd_signal": macd_signal, "macd_hist": macd_hist}, index=x.index)

    macd_df = grouped["Close"].apply(macd_apply)
    macd_df["macd"]        = macd_df["macd"] / df["Close"]
    macd_df["macd_signal"] = macd_df["macd_signal"] / df["Close"]
    macd_df["macd_hist"]   = macd_df["macd_hist"] / df["Close"]
    df = df.join(macd_df)


    # ------------------------------
    # 8. Stock vs Index Features
    # ------------------------------

    # relative return
    df["rel_ret"] = df["stock_ret_1d"] - df["index_ret_1d"]

    # rolling beta
    def rolling_beta(stock_ret, index_ret, win=20):
        cov = stock_ret.rolling(win).cov(index_ret)
        var = index_ret.rolling(win).var()
        return cov / var
    

    df["beta"] = grouped.apply(lambda g: rolling_beta(g["stock_ret_1d"], g["index_ret_1d"], window), include_groups=False)

    # rolling correlation
    df["corr"] = grouped.apply(lambda g: g["stock_ret_1d"].rolling(window).corr(g["index_ret_1d"]), include_groups=False)

    # excess volatility
    df["stock_vol"] = grouped["stock_ret_1d"].transform(lambda x: x.rolling(window).std())
    df["excess_vol"] = df["stock_vol"] - df["index_vol"]

    # ------------------------------
    # 9. Range-Based Volatility
    # ------------------------------
    df["parkinson_vol"] = grouped.apply(lambda g: np.sqrt(1/(4*np.log(2)) * (np.log(g["High"]/g["Low"]))**2), include_groups=False)

    df["gk_vol"] = grouped.apply(
        lambda g: np.sqrt(
            0.5*(np.log(g["High"]/g["Low"]))**2 -
            (2*np.log(2)-1)*(np.log(g["Close"]/g["Open"]))**2
        ),
        include_groups=False
    )
    df["vol_of_vol"] = grouped["stock_ret_1d"].transform(lambda x: x.rolling(window).std().rolling(window).std())

    # ------------------------------
    # 10. Distribution Shape
    # ------------------------------
    df["skew"] = grouped["stock_ret_1d"].transform(lambda x: x.rolling(window).skew())
    df["kurt"] = grouped["stock_ret_1d"].transform(lambda x: x.rolling(window).kurt())

    def downside_semivariance(x):
        neg = x[x < 0]
        return (neg**2).mean()

    df["dsv"] = grouped["stock_ret_1d"].transform(lambda x: x.rolling(window).apply(downside_semivariance, raw=False))

    # ------------------------------
    # 11. Volume Features
    # ------------------------------
    # df["vol_norm"] = grouped["Volume"].transform(lambda x: x / x.rolling(window).mean()) # drop it later
    df["vol_z"] = grouped["vol_norm"].transform(lambda x: (x - x.rolling(window).mean()) / x.rolling(window).std())

    df["vol_vol_ratio"] = grouped.apply(
        lambda g: (g["Volume"] / g["Volume"].rolling(window).mean()) / g["stock_ret_1d"].rolling(window).std(),
        include_groups=False
    )

    # df["obv"] = grouped.apply(lambda g: ta.OBV(g["Close"], g["Volume"]), include_groups=False) # drop it later
    df["obv_z"] = grouped["obv"].transform(lambda x: (x - x.rolling(window).mean()) / x.rolling(window).std())


    df["vwap_dev"] = grouped.apply(
        lambda g: (g["Close"] - (g["High"]+g["Low"]+g["Close"])/3) / ta.ATR(g["High"], g["Low"], g["Close"], timeperiod=14),
        include_groups=False
    )

    return df

In [61]:
modified = add_indicators(merged, window=window)

In [62]:
modified.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Symbol',
       'stock_ret_1d', 'vol_norm', 'obv', 'Index_Close', 'index_ret_1d',
       'index_vol', 'EMA_9', 'EMA_30', 'EMA_100', 'ema_9_ratio',
       'ema_30_ratio', 'ema_100_ratio', 'plus_di', 'minus_di', 'adx', 'rsi',
       'roc_3', 'roc_7', 'roc_14', 'roc_30', 'atr_3', 'atr_7', 'atr_14',
       'atr_30', 'stock_ret_3d', 'stock_ret_7d', 'stock_ret_14d',
       'stock_ret_30d', 'vol_3d', 'vol_7d', 'vol_14d', 'vol_30d',
       '3_days_high', '7_days_high', '14_days_high', '30_days_high', 'macd',
       'macd_signal', 'macd_hist', 'rel_ret', 'beta', 'corr', 'stock_vol',
       'excess_vol', 'parkinson_vol', 'gk_vol', 'vol_of_vol', 'skew', 'kurt',
       'dsv', 'vol_z', 'vol_vol_ratio', 'obv_z', 'vwap_dev'],
      dtype='object')

In [85]:
modified.iloc[388190:388200]

,Date,Open,High,Low,Close,Volume,Symbol,Index_Close,index_ret_1d,index_vol,...,parkinson_vol,gk_vol,vol_of_vol,skew,kurt,dsv,vol_z,vol_vol_ratio,obv,vwap_dev
388190,2025-09-05,676.200012,680.599976,666.400024,672.099976,582676.0,TATATECH,24741.000000,0.000271,0.006145,...,0.012663,0.014422,0.001229,0.393592,1.067420,0.000109,0.004972,4.778988e+07,27830458.0,-0.066653
388191,2025-09-08,674.049988,684.000000,672.799988,678.200012,527291.0,TATATECH,24773.150391,0.001299,0.006148,...,0.009915,0.011033,0.001242,0.271256,0.797880,0.000120,-0.196367,4.271245e+07,28357749.0,-0.009623
388192,2025-09-09,680.099976,689.099976,678.250000,679.549988,598002.0,TATATECH,24868.599609,0.003853,0.005733,...,0.009531,0.011211,0.001243,0.188982,1.273312,0.000113,0.364488,5.034172e+07,28955751.0,-0.201585
388193,2025-09-10,683.799988,711.900024,682.049988,704.849976,2612862.0,TATATECH,24973.099609,0.004202,0.005469,...,0.025725,0.023713,0.001381,0.592936,1.038663,0.000113,4.100646,1.826769e+08,31568613.0,0.350514
388194,2025-09-11,709.000000,709.000000,698.000000,702.099976,902219.0,TATATECH,25005.500000,0.001297,0.005352,...,0.009391,0.009236,0.001411,0.610032,0.998252,0.000114,0.484185,6.283449e+07,30666394.0,-0.063523
388195,2025-09-12,703.000000,707.299988,700.000000,702.099976,705641.0,TATATECH,25114.000000,0.004339,0.005313,...,0.006231,0.007293,0.001435,0.747345,1.275678,0.000114,0.058849,4.954515e+07,30666394.0,-0.072945
388196,2025-09-15,710.150024,720.750000,707.750000,711.450012,2115248.0,TATATECH,25069.199219,-0.001784,0.005349,...,0.010931,0.012820,0.001416,0.555694,0.992824,0.000125,2.363786,1.478366e+08,32781642.0,-0.128857
388197,2015-09-16,1092.031128,1096.305176,1086.068726,1090.043701,1756816.0,TCS,7899.149902,0.008947,0.018447,...,0.005634,0.006536,NaN,NaN,NaN,NaN,NaN,NaN,1756816.0,NaN
388198,2015-09-18,1096.304810,1105.280396,1085.192261,1090.427979,2455294.0,TCS,7981.899902,0.010476,0.018509,...,0.011015,0.012532,NaN,NaN,NaN,NaN,NaN,NaN,4212110.0,NaN
388199,2015-09-21,1089.893555,1092.928223,1082.713135,1090.342407,1108840.0,TCS,7977.100098,-0.000601,0.018448,...,0.005640,0.006635,NaN,NaN,NaN,NaN,NaN,NaN,3103270.0,NaN


In [50]:
modified.query("Symbol=='TATATECH'").head(10)

,Date,Open,High,Low,Close,Volume,Symbol,Index_Close,index_ret_1d,EMA_9,EMA_30,EMA_100,ema_9_ratio,ema_30_ratio,ema_100_ratio,plus_di,minus_di,adx
387758,2023-11-30,1170.154053,1365.179688,1170.154053,1280.343506,46579440.0,TATATECH,20133.150391,0.001819,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
387759,2023-12-01,1305.989502,1314.473022,1181.465576,1190.241699,18681410.0,TATATECH,20267.900391,0.006693,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
387760,2023-12-04,1190.241699,1203.308472,1122.519165,1162.986938,14740750.0,TATATECH,20686.800781,0.020668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
387761,2023-12-05,1162.986938,1191.558105,1145.775879,1162.206787,4422358.0,TATATECH,20855.099609,0.008136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
387762,2023-12-06,1172.104248,1177.418701,1151.626587,1158.940063,2735968.0,TATATECH,20937.699219,0.003961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
387763,2023-12-07,1163.279419,1169.178955,1150.651489,1156.209717,1624651.0,TATATECH,20901.150391,-0.001746,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
387764,2023-12-08,1159.427612,1189.364014,1158.696289,1162.889404,2528210.0,TATATECH,20969.400391,0.003265,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
387765,2023-12-11,1169.178955,1238.120483,1163.328125,1206.575073,8017469.0,TATATECH,20997.099609,0.001321,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
387766,2023-12-12,1222.811035,1231.635986,1199.407959,1202.577148,2426780.0,TATATECH,20906.400391,-0.004320,1186.996704,NaN,NaN,1.013126,NaN,NaN,NaN,NaN,NaN
387767,2023-12-13,1213.059692,1252.844971,1205.404907,1219.251709,4281933.0,TATATECH,20926.349609,0.000954,1193.447705,NaN,NaN,1.021621,NaN,NaN,NaN,NaN,NaN


In [63]:
df = modified.dropna().copy()

In [64]:
def check_if_uptrend(row):
    ema_check = 0
    dmi_check = 0
    if row['EMA_9'] > row['EMA_30'] > row['EMA_100']:
        ema_check = 1
    if row['plus_di'] > row['minus_di'] and row['adx'] > 30:
        dmi_check = 1
    if ema_check and dmi_check:
        return 1
    else:
        return -1

In [65]:
df["is_uptrend"] = df.apply(check_if_uptrend, axis=1)
df["signal"] = df["is_uptrend"].where(df["is_uptrend"] != df["is_uptrend"].shift(), 0)

In [66]:
def build_trades_df(df: pd.DataFrame) -> pd.DataFrame:
    # Ensure proper ordering
    df = df.sort_values(["Symbol", "Date"]).reset_index(drop=True)

    # Shift within symbol
    df["prev_signal"] = df.groupby("Symbol")["signal"].shift(fill_value=0)

    # Mark entry/exit rows
    entries = df.loc[(df["signal"] == 1) & (df["prev_signal"] != 1),
                     ["Symbol", "Date", "Close"]].rename(columns={"Date": "entry_date", "Close": "entry_price"})
    exits = df.loc[(df["signal"] == -1) & (df["prev_signal"] != -1),
                   ["Symbol", "Date", "Close"]].rename(columns={"Date": "exit_date", "Close": "exit_price"})

    # 🚨 Remove exits that occur before the first entry per symbol
    first_entries = entries.groupby("Symbol")["entry_date"].min()
    exits = exits.merge(first_entries, on="Symbol", how="left")
    exits = exits[exits["exit_date"] >= exits["entry_date"]].drop(columns=["entry_date"])

    # Assign trade_id per symbol (after cleaning exits)
    entries["trade_id"] = entries.groupby("Symbol").cumcount()
    exits["trade_id"]   = exits.groupby("Symbol").cumcount()

    # Pair entries with exits (only complete trades survive)
    trades = pd.merge(entries, exits, on=["Symbol", "trade_id"], how="inner").drop(columns=["trade_id"])

    return trades



trades_df = build_trades_df(df)

In [ ]:
df.columns

Index(['Symbol', 'entry_date', 'entry_price', 'exit_date', 'exit_price'], dtype='object')

In [68]:
df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Symbol',
       'stock_ret_1d', 'vol_norm', 'obv', 'Index_Close', 'index_ret_1d',
       'index_vol', 'EMA_9', 'EMA_30', 'EMA_100', 'ema_9_ratio',
       'ema_30_ratio', 'ema_100_ratio', 'plus_di', 'minus_di', 'adx', 'rsi',
       'roc_3', 'roc_7', 'roc_14', 'roc_30', 'atr_3', 'atr_7', 'atr_14',
       'atr_30', 'stock_ret_3d', 'stock_ret_7d', 'stock_ret_14d',
       'stock_ret_30d', 'vol_3d', 'vol_7d', 'vol_14d', 'vol_30d',
       '3_days_high', '7_days_high', '14_days_high', '30_days_high', 'macd',
       'macd_signal', 'macd_hist', 'rel_ret', 'beta', 'corr', 'stock_vol',
       'excess_vol', 'parkinson_vol', 'gk_vol', 'vol_of_vol', 'skew', 'kurt',
       'dsv', 'vol_z', 'vol_vol_ratio', 'obv_z', 'vwap_dev', 'is_uptrend',
       'signal'],
      dtype='object')

In [69]:
to_drop = [ 'Open', 'High', 'Low', 'Volume', 'EMA_9', 'EMA_30', 'EMA_100', 'is_uptrend', 'signal','obv','vol_norm']

In [70]:
trades_df["return"] = (trades_df["exit_price"] - trades_df["entry_price"])*100/trades_df["entry_price"]

In [71]:
enhanced_trades_df = trades_df.merge(df, left_on=["Symbol", "entry_date"], right_on=["Symbol", "Date"], how="left").drop(columns=["Date", *to_drop]).reset_index(drop=True)

In [72]:
enhanced_trades_df["result"] = np.where(enhanced_trades_df["return"] > 0, 1, 0)

In [73]:
enhanced_trades_df=enhanced_trades_df.merge(n200[["Symbol", "Industry"]], on="Symbol")

In [75]:
enhanced_trades_df

,Symbol,entry_date,entry_price,exit_date,exit_price,return,Close,stock_ret_1d,Index_Close,index_ret_1d,...,vol_of_vol,skew,kurt,dsv,vol_z,vol_vol_ratio,obv_z,vwap_dev,result,Industry
0,ABB,2016-04-01,1130.375488,2016-05-04,1072.873291,-5.087000,1130.375488,0.009983,7713.049805,-0.003276,...,0.005451,0.306493,0.117262,0.000249,1.528192,78.290163,2.057573,0.066625,0,Capital Goods
1,ABB,2017-04-11,1267.503418,2017-05-23,1273.588867,0.480113,1267.503418,0.029695,9237.000000,0.006050,...,0.002609,1.003524,0.934373,0.000101,-0.244947,71.118903,2.087195,0.314671,1,Capital Goods
2,ABB,2018-01-15,1330.367676,2018-02-19,1306.259766,-1.812124,1330.367676,0.025037,10741.549805,0.005645,...,0.001713,2.205441,6.596415,0.000050,1.770258,192.689483,2.686783,0.220293,0,Capital Goods
3,ABB,2018-09-06,1142.333374,2018-10-03,1253.569824,9.737652,1142.333374,0.021961,11536.900391,0.005224,...,0.002117,1.007868,0.699897,0.000115,-0.088891,78.644890,0.792242,0.116395,1,Capital Goods
4,ABB,2019-04-10,1217.739868,2019-05-09,1225.911133,0.671019,1217.739868,-0.019029,11584.299805,-0.007509,...,0.000901,1.079233,1.627410,0.000076,-0.588970,35.372286,1.555094,-0.154695,1,Capital Goods
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3664,ZYDUSLIFE,2023-04-18,504.088043,2023-05-02,505.113983,0.203524,504.088043,0.027485,17660.150391,-0.002637,...,0.002692,1.660169,4.146451,0.000011,2.308264,236.073903,1.461623,0.484949,1,Healthcare
3665,ZYDUSLIFE,2023-06-21,547.276001,2023-08-28,625.555908,14.303552,547.276001,0.007284,18856.849609,0.002134,...,0.000798,0.465201,0.199016,0.000092,-0.071581,79.290502,2.074164,0.092758,1,Healthcare
3666,ZYDUSLIFE,2023-08-29,623.238342,2023-08-30,619.441406,-0.609227,623.238342,-0.003705,19342.650391,0.001896,...,0.001165,0.265467,-0.563798,0.000125,-0.708708,67.572598,-0.570426,-0.152031,0,Healthcare
3667,ZYDUSLIFE,2023-11-09,604.007141,2024-04-12,948.886414,57.098541,604.007141,-0.020393,19395.300781,-0.002479,...,0.002258,1.254320,2.950709,0.000140,0.554319,92.705143,2.027133,-0.284898,1,Healthcare


In [76]:
regime

,Date,Date,kmeans_cluster,hmm_cluster
0,0,2015-03-17,2.0,1.0
1,1,2015-03-18,4.0,1.0
2,2,2015-03-19,4.0,1.0
3,3,2015-03-20,4.0,1.0
4,4,2015-03-21,4.0,1.0
...,...,...,...,...
3835,3835,2025-09-15,0.0,2.0
3836,3836,2025-09-16,0.0,2.0
3837,3837,2025-09-17,0.0,2.0
3838,3838,2025-09-18,0.0,2.0


In [81]:
enhanced_trades_df=enhanced_trades_df.merge(regime, left_on=["entry_date"], right_on=["Date"], how="left").drop(columns=["Date"], axis=1)

In [103]:
enhanced_trades_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3669 entries, 0 to 3668
Data columns (total 58 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Symbol         3669 non-null   object        
 1   entry_date     3669 non-null   datetime64[ns]
 2   entry_price    3669 non-null   float32       
 3   exit_date      3669 non-null   datetime64[ns]
 4   exit_price     3669 non-null   float32       
 5   return         3669 non-null   float32       
 6   Close          3669 non-null   float32       
 7   Index_Close    3669 non-null   float64       
 8   index_ret_1d   3669 non-null   float64       
 9   index_vol      3669 non-null   float64       
 10  ema_9_ratio    3669 non-null   float64       
 11  ema_30_ratio   3669 non-null   float64       
 12  ema_100_ratio  3669 non-null   float64       
 13  plus_di        3669 non-null   float64       
 14  minus_di       3669 non-null   float64       
 15  adx            3669 n

In [82]:
with open('../../data/enhanced_trades_df_v2.pkl', 'wb') as f:
    pickle.dump(enhanced_trades_df, f)

# with open('../../data/enhanced_trades_df_v2.pkl', 'rb') as f:
#     enhanced_trades_df = pickle.load(f)

In [83]:
enhanced_trades_df

,Symbol,entry_date,entry_price,exit_date,exit_price,return,Close,stock_ret_1d,Index_Close,index_ret_1d,...,kurt,dsv,vol_z,vol_vol_ratio,obv_z,vwap_dev,result,Industry,kmeans_cluster,hmm_cluster
0,ABB,2016-04-01,1130.375488,2016-05-04,1072.873291,-5.087000,1130.375488,0.009983,7713.049805,-0.003276,...,0.117262,0.000249,1.528192,78.290163,2.057573,0.066625,0,Capital Goods,2.0,1.0
1,ABB,2017-04-11,1267.503418,2017-05-23,1273.588867,0.480113,1267.503418,0.029695,9237.000000,0.006050,...,0.934373,0.000101,-0.244947,71.118903,2.087195,0.314671,1,Capital Goods,2.0,0.0
2,ABB,2018-01-15,1330.367676,2018-02-19,1306.259766,-1.812124,1330.367676,0.025037,10741.549805,0.005645,...,6.596415,0.000050,1.770258,192.689483,2.686783,0.220293,0,Capital Goods,2.0,0.0
3,ABB,2018-09-06,1142.333374,2018-10-03,1253.569824,9.737652,1142.333374,0.021961,11536.900391,0.005224,...,0.699897,0.000115,-0.088891,78.644890,0.792242,0.116395,1,Capital Goods,2.0,0.0
4,ABB,2019-04-10,1217.739868,2019-05-09,1225.911133,0.671019,1217.739868,-0.019029,11584.299805,-0.007509,...,1.627410,0.000076,-0.588970,35.372286,1.555094,-0.154695,1,Capital Goods,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3664,ZYDUSLIFE,2023-04-18,504.088043,2023-05-02,505.113983,0.203524,504.088043,0.027485,17660.150391,-0.002637,...,4.146451,0.000011,2.308264,236.073903,1.461623,0.484949,1,Healthcare,0.0,2.0
3665,ZYDUSLIFE,2023-06-21,547.276001,2023-08-28,625.555908,14.303552,547.276001,0.007284,18856.849609,0.002134,...,0.199016,0.000092,-0.071581,79.290502,2.074164,0.092758,1,Healthcare,0.0,2.0
3666,ZYDUSLIFE,2023-08-29,623.238342,2023-08-30,619.441406,-0.609227,623.238342,-0.003705,19342.650391,0.001896,...,-0.563798,0.000125,-0.708708,67.572598,-0.570426,-0.152031,0,Healthcare,0.0,2.0
3667,ZYDUSLIFE,2023-11-09,604.007141,2024-04-12,948.886414,57.098541,604.007141,-0.020393,19395.300781,-0.002479,...,2.950709,0.000140,0.554319,92.705143,2.027133,-0.284898,1,Healthcare,0.0,2.0
